In [50]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [51]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.2.1


In [52]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [53]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
"""
    Parameters
    ----------
    problem_formulation_id : int {0, ..., 5}
                             problem formulations differ with respect to the objectives
                             0: Total cost, and casualties
                             1: Expected damages, costs, and casualties
                             2: expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
                             3: costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
                             4: Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
                             5: disaggregate over time and space

    Notes
    -----
    problem formulations 4 and 5 rely on ArrayOutcomes and thus cannot straightforwardly
    be used in optimizations

    """
problem_formulation = 3
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)

In [54]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [55]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [56]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [57]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2, policies=4)

[MainProcess/INFO] performing 2 scenarios * 4 policies * 1 model(s) = 8 experiments
[MainProcess/INFO] performing experiments sequentially
100%|████████████████████████████████████████████| 8/8 [00:09<00:00,  1.14s/it]
[MainProcess/INFO] experiments finished


In [58]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,60,179.099195,1.0,0.896890,347.547458,1.5,0.510004,112.112381,1.0,0.045465,263.895607,1.0,0.374647,322.696947,1.0,0.370381,3.5,2.5,3.5,0,1,0,0,0,1,0,0,1,1,1,1,1,0,1,5,10,7,4,10,5,8,6,10,8,10,0,10,9,4,2,182,178,dikesnet
1,92,322.211426,10.0,0.276814,105.130837,10.0,0.466205,310.920201,10.0,0.513448,123.999610,1.5,0.687492,164.288054,10.0,0.620377,2.5,4.5,1.5,0,1,0,0,0,1,0,0,1,1,1,1,1,0,1,5,10,7,4,10,5,8,6,10,8,10,0,10,9,4,2,183,178,dikesnet
2,60,179.099195,1.0,0.896890,347.547458,1.5,0.510004,112.112381,1.0,0.045465,263.895607,1.0,0.374647,322.696947,1.0,0.370381,3.5,2.5,3.5,0,0,1,1,0,0,1,0,0,0,0,1,0,1,1,10,0,4,2,5,4,2,8,8,3,6,4,6,7,6,4,182,179,dikesnet
3,92,322.211426,10.0,0.276814,105.130837,10.0,0.466205,310.920201,10.0,0.513448,123.999610,1.5,0.687492,164.288054,10.0,0.620377,2.5,4.5,1.5,0,0,1,1,0,0,1,0,0,0,0,1,0,1,1,10,0,4,2,5,4,2,8,8,3,6,4,6,7,6,4,183,179,dikesnet
4,60,179.099195,1.0,0.896890,347.547458,1.5,0.510004,112.112381,1.0,0.045465,263.895607,1.0,0.374647,322.696947,1.0,0.370381,3.5,2.5,3.5,1,0,0,1,1,1,1,1,1,1,0,0,0,1,0,1,2,1,8,1,0,3,4,0,1,1,7,1,0,0,0,182,180,dikesnet
5,92,322.211426,10.0,0.276814,105.130837,10.0,0.466205,310.920201,10.0,0.513448,123.999610,1.5,0.687492,164.288054,10.0,0.620377,2.5,4.5,1.5,1,0,0,1,1,1,1,1,1,1,0,0,0,1,0,1,2,1,8,1,0,3,4,0,1,1,7,1,0,0,0,183,180,dikesnet
6,60,179.099195,1.0,0.896890,347.547458,1.5,0.510004,112.112381,1.0,0.045465,263.895607,1.0,0.374647,322.696947,1.0,0.370381,3.5,2.5,3.5,1,1,1,0,1,0,0,1,0,0,1,0,1,0,0,2,5,10,9,4,10,7,2,3,10,5,10,5,3,10,3,182,181,dikesnet
7,92,322.211426,10.0,0.276814,105.130837,10.0,0.466205,310.920201,10.0,0.513448,123.999610,1.5,0.687492,164.288054,10.0,0.620377,2.5,4.5,1.5,1,1,1,0,1,0,0,1,0,0,1,0,1,0,0,2,5,10,9,4,10,7,2,3,10,5,10,5,3,10,3,183,181,dikesnet


In [59]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.792709e+08,0.0,2.622946e+08,0.000000,1.491391e+08,0.000000,4.277965e+07,0.000000,1.956990e+08,0.000000,1.208900e+09,0.000000
1,2.792709e+08,0.0,2.622946e+08,0.000000,1.491391e+08,0.000000,4.277965e+07,0.000000,1.956990e+08,0.000000,1.208900e+09,0.000000
2,1.620628e+08,0.0,1.774148e+08,0.000510,3.015865e+08,0.045152,3.549332e+07,0.000000,1.566716e+08,0.000000,9.665000e+08,13692.112851
3,1.620628e+08,0.0,1.769706e+08,0.000352,1.142189e+08,0.000000,3.549332e+07,0.000000,1.566716e+08,0.000000,9.665000e+08,190.085331
4,1.073855e+08,0.0,1.313171e+08,0.000000,7.087625e+07,0.036562,3.150673e+07,0.001975,8.639206e+07,0.062559,1.207400e+09,0.000000
5,1.073855e+08,0.0,1.313171e+08,0.000000,5.340163e+07,0.000000,2.848681e+07,0.000204,3.413092e+07,0.008328,1.207400e+09,0.000000
6,2.153055e+08,0.0,3.162317e+08,0.000000,9.697269e+07,0.000528,6.961610e+07,0.000000,1.486985e+08,0.000000,8.796000e+08,103.931976
7,2.153055e+08,0.0,3.162317e+08,0.000000,9.500985e+07,0.000000,6.961610e+07,0.000000,1.486985e+08,0.000000,8.796000e+08,0.000000


In [60]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations
# policy 4 doe niks
'''
A1 = Doesburg Upstream
A2 = Cortenoever Upmidstream
A3 = Zutphen Midstream
A4 = Gorssel Downmidstream
A5 = Deventer Downstream
0_RFR = Project Olburgen
1_RFR = Project Havikerwaard
2_RFR = project Tichelbeekse
3_RFR = Project Welsummer
4_RFR = Obstakelsverwijderen
dan 0 of 1 of 2 is de timestep waarin het wordt geddaan
'''



def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 8}
        )
    ),
        Policy(
        "policy x",
        **dict(
            get_do_nothing_dict(),
            **{"3_RfR 0": 1, "3_RfR 1": 1, "3_RfR 2": 1, "4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1}
        )
    ),
    
    # Policy(
    #     "policy 2",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 3",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 4",
    #     **dict(
    #         get_do_nothing_dict(),
            
    #     )
    # ),
]

In [61]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 20
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] performing 20 scenarios * 2 policies * 1 model(s) = 40 experiments
[MainProcess/INFO] performing experiments sequentially
100%|██████████████████████████████████████████| 40/40 [00:47<00:00,  1.19s/it]
[MainProcess/INFO] experiments finished


In [62]:
experiments, outcomes = results

In [63]:
# only works because we have scalar outcomes
outcomes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
outcomes=pd.DataFrame(outcomes)
outcomes

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.000000e+00,0.000000,4.726949e+08,0.385941,5.765224e+07,0.089664,1.495740e+08,0.054356,0.000000e+00,0.000000,3.459000e+08,0.0
1,0.000000e+00,0.000000,5.556493e+07,0.056329,0.000000e+00,0.000000,1.390487e+07,0.007639,0.000000e+00,0.000000,3.459000e+08,0.0
2,0.000000e+00,0.000000,2.793722e+07,0.032753,0.000000e+00,0.000000,2.013076e+06,0.001381,1.002436e+08,0.115666,3.459000e+08,0.0
3,0.000000e+00,0.000000,0.000000e+00,0.000000,5.458987e+07,0.112299,7.469458e+07,0.041041,0.000000e+00,0.000000,3.459000e+08,0.0
4,0.000000e+00,0.000000,3.358317e+06,0.003682,0.000000e+00,0.000000,6.695804e+06,0.004335,7.151193e+08,0.706801,3.459000e+08,0.0
5,0.000000e+00,0.000000,6.253449e+08,0.666117,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,3.459000e+08,0.0
6,0.000000e+00,0.000000,2.239226e+07,0.022165,1.503130e+07,0.027486,5.836394e+07,0.029204,5.280615e+08,0.483289,3.459000e+08,0.0
7,0.000000e+00,0.000000,1.807599e+08,0.167708,0.000000e+00,0.000000,1.363482e+07,0.006640,1.321457e+08,0.123785,3.459000e+08,0.0
8,4.064790e+08,0.288236,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,3.459000e+08,0.0
9,0.000000e+00,0.000000,1.188818e+07,0.012047,0.000000e+00,0.000000,1.665208e+07,0.009394,0.000000e+00,0.000000,3.459000e+08,0.0


In [67]:
print(np.mean(outcomes["A.2 Total Costs"]))
print(np.mean(outcomes["A.1_Expected Number of Deaths",][0:20]))
print(np.mean(outcomes["A.2_Expected Number of Deaths", 0:20][0:20]))
print(np.mean(outcomes["A.3_Expected Number of Deaths", 0:20][0:20]))

107725843.58373079


KeyError: ('A.1_Expected Number of Deaths',)